# Random Forest Classification Consulting Project Solutions

## Moje Rozwiązanie

### 1. Import koniecznych pakietów oraz wstępne przygotowanie danych.

Zaczynamy od importowania pakietu findspark, aby w dalszej kolejności umożliwić import Sparka i jego klas.

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dog_food').getOrCreate()

Import danych z pliku CSV.

In [5]:
data = spark.read.csv('dog_food.csv', inferSchema=True, header=True)

In [6]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [10]:
data.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|                 A|                 B|                 C|                 D|            Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|               490|               490|               490|               490|                490|
|   mean|  5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
| stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
|    min|                 1|                 1|               5.0|                 1|                0.0|
|    max|                10|                10|              14.0|                10|                1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+



Dane sprawiają wrażenie kompletnych. Wszystkie kolumny mają równą ilość wartości. Nie ma konieczności usupełniania brakujących danych. Label, którym jest 'Spoiled' jest numeryczny tak jak pozostałe kolumny. Importujemy klasę odpowiedzialną za wstępną obróbkę danych - utworzenie kolumn 'features' oraz 'label'.

In [12]:
from pyspark.ml.feature import VectorAssembler

In [13]:
features_col = ['A', 'B', 'C', 'D']

In [14]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='features')

In [15]:
output = assembler.transform(data)

In [16]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [18]:
final_data = output.select('features', 'Spoiled')

Naszym zadnaiem jet wykrycie, który z konserwantów ma najwiekszywpływ na psucie się jedzenia. Z tego względu nie ma konieczności podziału danych na zestaw treningowy i testowy.

### 2. Tworzenie i trening klasyfikatora

Importujemy RandomForestClassifier

In [25]:
from pyspark.ml.classification import RandomForestClassifier

In [26]:
rfc = RandomForestClassifier(labelCol='Spoiled', featuresCol='features', numTrees=100)

In [27]:
rfc_model = rfc.fit(final_data)

In [28]:
rfc_preds = rfc_model.transform(test_data)

In [29]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0233, 1: 0.0204, 2: 0.9332, 3: 0.0231})

Z powyższych danych wynika, że na ewentualne psucie się jedzenia ma wpływ konserwant C.